In [90]:
import tensorflow as tf
from tensorflow.keras.models import load_model # type: ignore
import pickle
import numpy as np
import pandas as pd


In [91]:
### Load the trained model, scaler pickle and onehot encodeing pickle
model = load_model('model.h5')

## load the encoder and scaler
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('onehot_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)


In [92]:
input_data = {
  "CreditScore": 400,
  "Gender": "Male",
  "Age": 25,
  "Tenure": 2,
  "Balance": 111212120.03,
  "NumOfProducts": 1,
  "HasCrCard": 0,
  "IsActiveMember": 1,
  "EstimatedSalary": 11344.45,
  "Geography": "Germany"
}

In [93]:
#Implementing one hot encoding for the geography column
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns = label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/Users/ankitpokhrel/Downloads/All projects/Churn_ANN/myenv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [94]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography
0,400,Male,25,2,1.112121e+08,1,0,1,11344.45,Germany


In [95]:
#Now we will drop the original geography column and add the encoded columns
input_df = input_df.drop(columns = ['Geography'])
input_df = pd.concat([input_df,geo_encoded_df],axis = 1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,400,Male,25,2,1.112121e+08,1,0,1,11344.45,0.0,1.0,0.0


In [96]:
#Now let's encode the Gender and replace the encoded value in gender
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,400,1,25,2,1.112121e+08,1,0,1,11344.45,0.0,1.0,0.0


In [97]:
### Scaling the input data
input_data_scaled = scaler.transform(input_df)
input_data_scaled


array([[-2.61153137e+00,  9.13247552e-01, -1.32129295e+00,
        -1.04241787e+00,  1.77939761e+03, -9.16687674e-01,
        -1.54035103e+00,  9.74816989e-01, -1.54892351e+00,
        -9.98501123e-01,  1.72572313e+00, -5.76388018e-01]])

In [98]:
### PREDICTION ###
prediction = model.predict(input_data_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


array([[0.75290453]], dtype=float32)

In [99]:
prediction_proba = prediction[0][0]

In [100]:
#Let's check the prediction probability
if prediction_proba > 0.5:
    print("The customer is likely to leave the bank")
else:
    print("The customer is likely to stay with the bank")


The customer is likely to leave the bank
